In [1]:
from pandas import (
    read_excel,
    read_csv,
    DataFrame,
    merge,
)
from numpy import (
    std,
)

In [2]:
df = read_csv(
    'standard_oct20frl.csv',
    low_memory=False,
    #sep='\t',
    #error_bad_lines=False,
)
must_games = False

In [3]:
df.head()

,ID,Name,Fed,Sex,Title,WomanTitle,OtherTitle,FOA,Rating,Games,K,Birthday,Flag,Included
0,25121731,A C J John,IND,M,NaN,NaN,NaN,NaN,1063,0,40,1987,i,1
1,35077023,A Chakravarthy,IND,M,NaN,NaN,NaN,NaN,1151,0,40,1986,i,1
2,10207538,"A E M, Doshtagir",BAN,M,NaN,NaN,NaN,NaN,1840,0,40,1974,i,0
3,10680810,"A hamed Ashraf, Abdallah",EGY,M,NaN,NaN,NaN,NaN,1728,0,40,2001,i,0
4,5716365,"A Hamid, Harman",MAS,M,NaN,NaN,NI,NaN,1325,0,40,1970,i,0


In [4]:
# df.to_csv('standard_oct20frl.csv', index=False)

In [5]:
if must_games:
    df = df[df['Games'] > 0]
df = df.sort_values(
    by=[
        'Rating',
        'Games',
    ],
)
df = df.reset_index(drop=True)

In [6]:
df.shape

(361760, 14)

In [7]:
df.head()

,ID,Name,Fed,Sex,Title,WomanTitle,OtherTitle,FOA,Rating,Games,K,Birthday,Flag,Included
0,25769367,Abhimanyu C B,IND,M,NaN,NaN,NaN,NaN,1001,0,40,2005,i,0
1,25625675,Abimuhesh K G,IND,M,NaN,NaN,NaN,NaN,1001,0,40,2004,i,0
2,24538930,"Acebes Moran, Roberto",ESP,M,NaN,NaN,NaN,NaN,1001,0,40,2006,NaN,0
3,1555529,"Aglen, John",NOR,M,NaN,NaN,NaN,NaN,1001,0,40,1981,NaN,0
4,25786458,Agnihotri Viraj,IND,M,NaN,NaN,NaN,NaN,1001,0,40,2010,i,0


In [8]:
dfs = {}
keys = [
    'woman',
    'man',
]
#dfs['bd'] = df[df['Fed'] == 'BAN']
dfs['woman'] = df[df['Sex'] == 'F']
dfs['man'] = df[df['Sex'] == 'M']

In [9]:
dfs['man'].head()

,ID,Name,Fed,Sex,Title,WomanTitle,OtherTitle,FOA,Rating,Games,K,Birthday,Flag,Included
0,25769367,Abhimanyu C B,IND,M,NaN,NaN,NaN,NaN,1001,0,40,2005,i,0
1,25625675,Abimuhesh K G,IND,M,NaN,NaN,NaN,NaN,1001,0,40,2004,i,0
2,24538930,"Acebes Moran, Roberto",ESP,M,NaN,NaN,NaN,NaN,1001,0,40,2006,NaN,0
3,1555529,"Aglen, John",NOR,M,NaN,NaN,NaN,NaN,1001,0,40,1981,NaN,0
4,25786458,Agnihotri Viraj,IND,M,NaN,NaN,NaN,NaN,1001,0,40,2010,i,0


In [10]:
for key in dfs:
    dfs[key] = dfs[key].sort_values(
        by=[
            'Fed',
        ],
    )
    print(key, dfs[key].shape[0])

woman 38149
man 323611


In [11]:
dfs['woman'].head()

,ID,Name,Fed,Sex,Title,WomanTitle,OtherTitle,FOA,Rating,Games,K,Birthday,Flag,Included
64203,11700742,"Hamrah, Lidaa",AFG,F,NaN,NaN,DI,NaN,1284,0,40,1996,wi,0
210460,1039830,"Van Zutphen, Amy",AHO,F,WCM,WCM,NaN,NaN,1751,0,40,1985,wi,0
35634,7500645,"Koenraad, Arona",AHO,F,NaN,NaN,NaN,NaN,1171,0,40,2003,w,0
123396,7500610,"Marcos, Taqesyah",AHO,F,NaN,NaN,NaN,NaN,1493,0,40,2004,wi,0
267169,7500394,"Delemarre, Maria Elisabeth",AHO,F,NaN,NaN,NaN,NaN,1917,0,40,1975,wi,0


In [12]:
print('Country wise')
for key in keys:
    column = key+'_country'
    dfs[column] = dfs[key].groupby(
        by=[
            'Fed',
        ],
        as_index=False,
    )[['Rating']].agg(lambda x: list(x))
    dfs[column]['Rating'] = dfs[column]['Rating'].apply(lambda x: sorted(x))
    dfs[column]['count'] = dfs[column]['Rating'].apply(lambda x: len(x))
    dfs[column]['total'] = dfs[column]['Rating'].apply(lambda x: sum(x))
    dfs[column]['average'] = dfs[column]['total']/dfs[column]['count']
    dfs[column]['std'] = dfs[column]['Rating'].apply(lambda x: std(x))
    dfs[column] = round(dfs[column], 2)
    dfs[column] = dfs[column].sort_values(
        by=[
            'count',
            'average',
            'Fed',
        ],
    )
    dfs[column] = dfs[column].reset_index(drop=True)
    print(key, dfs[column].shape[0])

Country wise
woman 169
man 192


In [13]:
dfs['woman_country'].head(10)

,Fed,Rating,count,total,average,std
0,SLE,[1008],1,1008,1008.0,0.0
1,LIE,[1279],1,1279,1279.0,0.0
2,AFG,[1284],1,1284,1284.0,0.0
3,JCI,[1351],1,1351,1351.0,0.0
4,STP,[1434],1,1434,1434.0,0.0
5,CPV,[1858],1,1858,1858.0,0.0
6,CIV,"[1043, 1343]",2,2386,1193.0,150.0
7,CMR,"[1280, 1317]",2,2597,1298.5,18.5
8,ISV,"[1451, 1622]",2,3073,1536.5,85.5
9,GCI,"[1834, 1918]",2,3752,1876.0,42.0


In [14]:
dfs['woman_country'].describe()

,count,total,average,std
count,169.000000,1.690000e+02,169.000000,169.000000
mean,225.733728,3.406437e+05,1491.271243,244.348757
std,594.720828,8.927609e+05,171.021332,94.843053
min,1.000000,1.008000e+03,1008.000000,0.000000
25%,12.000000,1.790300e+04,1361.920000,189.640000
50%,47.000000,7.113700e+04,1475.360000,270.510000
75%,203.000000,3.116410e+05,1613.590000,314.000000
max,5851.000000,9.165044e+06,2000.750000,414.670000


In [15]:
dfs['man_country'].head(10)

,Fed,Rating,count,total,average,std
0,CAY,[1627],1,1627,1627.00,0.00
1,Ind,[1727],1,1727,1727.00,0.00
2,CAM,"[2165, 2335]",2,4500,2250.00,85.00
3,GAB,"[1688, 1803, 2142]",3,5633,1877.67,192.72
4,CAF,"[1387, 1483, 1677, 1679]",4,6226,1556.50,126.15
5,IVB,"[1438, 1616, 1723, 2220]",4,6997,1749.25,290.23
6,ANT,"[1049, 1208, 1209, 1283, 1337, 1418]",6,7504,1250.67,115.99
7,SLE,"[1441, 1590, 1818, 1832, 1899, 1964]",6,10544,1757.33,182.62
8,ERI,"[1712, 1778, 1792, 1804, 1820, 2017]",6,10923,1820.50,94.23
9,BHU,"[1158, 1324, 1408, 1460, 1487, 1568, 1661]",7,10066,1438.00,152.04


In [16]:
dfs['man_country'].describe()

,count,total,average,std
count,192.000000,1.920000e+02,192.000000,192.000000
mean,1685.473958,2.827598e+06,1701.182656,265.358802
std,4413.928589,7.276773e+06,145.298046,77.164382
min,1.000000,1.627000e+03,1250.670000,0.000000
25%,42.750000,6.648675e+04,1615.095000,212.190000
50%,292.500000,4.934800e+05,1714.760000,277.010000
75%,1066.000000,1.956842e+06,1804.102500,318.010000
max,29602.000000,5.133241e+07,2250.000000,408.670000
